<p style="font-weight:bold;"> <span style="font-size: 36px"> Financial Performance Specifications </span> </p>

The aim of this notebook is to document the Financial Performance reporting system. The process is pictorially explained in the following flowchart. 

<a name="TOC"></a>
<center><img src="../Images/SpecificationsPL.PNG" width="45%" style="float:center;">

Throughout this notebook we adopt the following variable notation:  we use the words *computed* vs. *expected* to differentiate the results provided by the application vs. this test notebook, respectively.

In the following the Dimensions, Parameters, Nominal Cashflows, and Actuals are imported from the corresponding csv files in <code>Initialization/Systemorph/</code>

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

---

# Process Data

<a id='year-and-period'></a>
## Current Period, Reporting Node and Currency Type

Please select the target period (i.e. year and month) and Reporting Node based on the imported data:

In [ ]:
var year          = 2021                    ;
var month         = 3                       ;
var reportingNode = "CH"                    ;

and the type of Currency among <code>Contractual</code>, <code>Functional</code>, and <code>Group</code>

In [ ]:
var currencyType  = CurrencyType.Functional ;

Import the Report definitions, initialize the Workspace and the Test Suite:

In [ ]:
Workspace.InitializeFrom(DataSource);

In [ ]:
var period        = (year, month)                                                        ;
var reportStorage = new ReportStorage(Workspace, Report)                                 ;
var Test          = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

In [ ]:
await reportStorage.InitializeReportIndependentCacheAsync();
await reportStorage.InitializeAsync(period, reportingNode, null, currencyType);

## View list of Identities

Based on the Cashflows imported, the list of all possible identities, i.e. combinations of valid Contractual and Functional Currencies, Liability Types, (Re)Insurance and Oci, is contructed and reported here below:

In [ ]:
var identities = reportStorage.GetIdentities(period, reportingNode, null, currencyType);
identities

<br/><br/>
<a id='report'></a>
# Report

Please select here below the target Data Node for executing the tests below:

In [ ]:
var dataNode = "DT1.1";

The Group of Contract selected is:

In [ ]:
var groupOfContract = (await Workspace.Query<GroupOfContract>().Where(x => x.SystemName == dataNode).ToArrayAsync()).FirstOrDefault();
groupOfContract

where 
- the **OciType** can be <code>Default</code> for contracts with non-zero OCI, and <code>null</code> otherwise, 
- the **Liability Type** can be <code>LRC</code> for Liability for Remaining Coverage, and <code>LIC</code> for Liability of Incurred Claims,
- the **Partner** can be <code>null</code> for Insurance contracts, or equal to the Partner System Name for Reinsurance contracts.

The summary of the Financial Performance report is shown below

In [ ]:
Report.ForDataCube( Test.GetScopes<FinancialPerformance>(identities).Aggregate().FinancialPerformance.Filter(("GroupOfContract", dataNode)) )
      .WithQuerySource(DataSource)
      .SliceRowsBy("VariableType", "EstimateType")
      .SliceColumnsBy(CurrencyGrouper(currencyType), "LiabilityType","GroupOfContract")
      .WithGridOptions( o => o.WithDefaultColumn( c => c.WithWidth(260) ) with {GroupDefaultExpanded = 3, Height = 900, OnGridReady = null} )
      .ToReport()

<br/><br/>
<a name='inspection'></a>
# Inspection

## Fulfillment Cashflows

The Fulfillment Cashflow (FCF) corresponds to the sum of the Best Estimate [Present Value](https://portal.stage.systemorph.cloud/project/ifrs17ce/env/dev/Import/ImportScopeCalculation#current-and-locked) (PV) and the [Risk Adjustment](https://portal.stage.systemorph.cloud/project/ifrs17ce/env/dev/Import/ImportScopeCalculation#risk-adjustment) (RA), both summed over Amounty Types, and both discounted with the Locked-In curve in the BBA valuation approach:

$$
\text{FCF}(\text{AoC}) =  \text{PV Locked}(\text{AoC})
\bigg|_{\substack{\text{Non Attributable} \\ \text{Amount Types} \\ \text{excluded}}}  
+ \text{RA Locked}(\text{AoC}) ~.
$$

### Non Financial

The computed value for the non financial contribution to the FCF is

In [ ]:
var computedDeltaFCF_nonFinancial = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                                        .FpNonFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaFCF_nonFinancial

This value can be independently computed starting from the full list of FCFs

In [ ]:
var FCFs = Test.GetScopes<Fcf>(identities).Aggregate().Fcf.Filter(("GroupOfContract", dataNode));

In [ ]:
var deltaFCF = FCFs.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
               FCFs.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [ ]:
deltaFCF.Aggregate().Value

In [ ]:
var expectedDeltaFCF_nonFinancial = deltaFCF.Filter(("VariableType", "!IA"), 
                                                    ("VariableType", "!YCU"), 
                                                    ("VariableType", "!CRU"), 
                                                    ("VariableType", "!FX") ).Aggregate().Value;

In [ ]:
expectedDeltaFCF_nonFinancial

For consistency, this term is taken with the minus sign

In [ ]:
expectedDeltaFCF_nonFinancial.CheckEquality( -computedDeltaFCF_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial contribution to the Fulfillment Cashflows (FCF) is

In [ ]:
var computedDeltaFCF_Financial = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                                     .FpFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaFCF_Financial

The independent recalculation of this value reads

In [ ]:
var expectedDeltaFCF_Financial = (deltaFCF.Filter(("VariableType", "IA")) +
                                  deltaFCF.Filter(("VariableType", "YCU")) +
                                  deltaFCF.Filter(("VariableType", "CRU")) ).Aggregate().Value;

In [ ]:
expectedDeltaFCF_Financial

taking the minus sign into account

In [ ]:
expectedDeltaFCF_Financial.CheckEquality( -computedDeltaFCF_Financial ).Should().Be(true);

### FX

The computed value for the FX contribution to the Fulfillment Cashflows (FCF) is

In [ ]:
var computedDeltaFCF_Fx = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                                     .FpFx.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaFCF_Fx

The independent recalculation of this value reads

In [ ]:
var expectedDeltaFCF_Fx = deltaFCF.Filter(("VariableType", "FX")).Aggregate().Value;

In [ ]:
expectedDeltaFCF_Fx

taking the minus sign into account

In [ ]:
expectedDeltaFCF_Fx.CheckEquality( -computedDeltaFCF_Fx ).Should().Be(true);

<a id='oci'></a>
### Other Comprehensive Income

The Other Comprehensive Income (OCI) term is by definition the sum of the financial contributions not related to the insurance sector.

The <code>Default</code> method to compute the OCI term is by subtracting the FCF computed with the *Locked-In* rates and the FCF computed with the *Current* rates. In this way the non insurance financial volatiliy of the economic input is relegated to the OCI. 

In [ ]:
var computedDeltaFCF_OCI = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                               .OciFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaFCF_OCI

The OCI contribution is zero when the OCI option is disabled for the [target Group of Contract](#report).

Here below we follow the steps to calculate independently the OCI contribution 

In [ ]:
var DeltaFCF_Locked = deltaFCF.Aggregate().Value;

In [ ]:
var FCFs_current = Test.GetScopes<CurrentFcf>(identities).Aggregate().CurrentFcf.Filter(("GroupOfContract", dataNode));

In [ ]:
var DeltaFCF_Current = FCFs_current.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                       FCFs_current.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [ ]:
var expectedDeltaFCF_OCI = DeltaFCF_Locked - DeltaFCF_Current.Aggregate().Value;

In [ ]:
DeltaFCF_Locked

In [ ]:
DeltaFCF_Current.Aggregate().Value

In [ ]:
expectedDeltaFCF_OCI

In [ ]:
expectedDeltaFCF_OCI.CheckEquality( computedDeltaFCF_OCI ).Should().Be(true);

### FX Other Comprehensive Income

The FX impact on the Other Comprehensive Income is calculated using the same formulas defined [above](#oci) but is presented separately in the Financial Performance.

The computed value for the FX contribution to the Other Comprehensive Income (OCI) is

In [ ]:
var computedFxFCF_OCI = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                               .OciFx.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedFxFCF_OCI

The OCI contribution is zero when the OCI option is disabled for the [target Group of Contract](#report).

Here below we follow the steps to calculate independently the FX OCI contribution 

In [ ]:
var FxFCF_Locked = deltaFCF.Filter(("VariableType", AocTypes.FX)).Aggregate().Value;

In [ ]:
var FxFCF_Current = FCFs_current.Filter(("VariableType", AocTypes.FX)).Aggregate().Value;

In [ ]:
var expectedFxFCF_OCI = FxFCF_Locked - FxFCF_Current;

In [ ]:
FxFCF_Locked

In [ ]:
FxFCF_Current

In [ ]:
expectedFxFCF_OCI

In [ ]:
expectedFxFCF_OCI.CheckEquality( computedFxFCF_OCI ).Should().Be(true);

## Contractual Service Margin

### Non Financial

The computed value for the non financial change of the CSM is

In [ ]:
var computedDeltaCSM_nonFinancial = Test.GetScopes<CsmChangeInEstimate>(identities).Aggregate()
                                        .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaCSM_nonFinancial

This value can be independently computed starting from the full CSM data from which the delta can be computed:

In [ ]:
var CSM = Test.GetScopes<Csm>(identities).Aggregate().Csm.Filter(("GroupOfContract", dataNode));

In [ ]:
var deltaCSM = CSM.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
               CSM.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [ ]:
deltaCSM.Aggregate().Value

The non Financial contribution is calculated subtracting the Amortization and the Interest Accretion terms:

In [ ]:
var expectedDeltaCSM_nonFinancial = deltaCSM.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX")).Aggregate().Value;

In [ ]:
expectedDeltaCSM_nonFinancial

In [ ]:
expectedDeltaCSM_nonFinancial.CheckEquality( -computedDeltaCSM_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the CSM is

In [ ]:
var computedDeltaCSM_Financial = Test.GetScopes<CsmChangeInEstimate>(identities).Aggregate()
                                     .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaCSM_Financial

This value can be independently computed starting from the $\Delta\text{CSM}$, and considering only Interest Accretion, Yield Curve Update and Credit Risk Update terms

In [ ]:
var expectedDeltaCSM_Financial = ( deltaCSM.Filter(("VariableType", "IA")) +
                                   deltaCSM.Filter(("VariableType", "YCU")) +
                                   deltaCSM.Filter(("VariableType", "CRU")) )
                                 .Aggregate().Value;

In [ ]:
expectedDeltaCSM_Financial

In [ ]:
expectedDeltaCSM_Financial.CheckEquality( -computedDeltaCSM_Financial ).Should().Be(true);

### FX

The computed value for the FX change of the CSM is

In [ ]:
var computedDeltaCSM_Fx = Test.GetScopes<CsmChangeInEstimate>(identities).Aggregate()
                                     .Fx.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaCSM_Fx

This value can be independently computed starting from the $\Delta\text{CSM}$, and considering only the FX term

In [ ]:
var expectedDeltaCSM_Fx = deltaCSM.Filter(("VariableType", AocTypes.FX)).Aggregate().Value;

In [ ]:
expectedDeltaCSM_Fx

In [ ]:
expectedDeltaCSM_Fx.CheckEquality( -computedDeltaCSM_Fx ).Should().Be(true);

## Loss Component

### Non Financial

The computed value for the non financial change of the LC is

In [ ]:
var computedDeltaLC_nonFinancial = Test.GetScopes<LcChangeInEstimate>(identities).Aggregate()
                                       .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaLC_nonFinancial

This value can be independently computed starting from the full CSM data from which the delta can be computed:

In [ ]:
var LC = Test.GetScopes<Lc>(identities).Aggregate().Lc.Filter(("GroupOfContract", dataNode));

In [ ]:
var deltaLC = LC.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
              LC.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [ ]:
deltaLC.Aggregate().Value

The non Financial contribution is calculated subtracting the Amortization, Interest Accretion, Yield Curve Update, Credit Risk Update, Fx terms:

In [ ]:
var expectedDeltaLC_nonFinancial = deltaLC.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX")).Aggregate().Value;

In [ ]:
expectedDeltaLC_nonFinancial

In [ ]:
expectedDeltaLC_nonFinancial.CheckEquality( -computedDeltaLC_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the LC is

In [ ]:
var computedDeltaLC_Financial = Test.GetScopes<LcChangeInEstimate>(identities).Aggregate()
                                    .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaLC_Financial

This value can be independently computed starting from the $\Delta\text{LC}$, and considering only the Interest Accretion, Yield Curve Update, Credit Risk Update terms

In [ ]:
var expectedDeltaLC_Financial = ( deltaLC.Filter(("VariableType", "IA")) +
                                  deltaLC.Filter(("VariableType", "YCU")) +
                                  deltaLC.Filter(("VariableType", "CRU")) )
                                .Aggregate().Value;

In [ ]:
expectedDeltaLC_Financial

In [ ]:
expectedDeltaLC_Financial.CheckEquality( -computedDeltaLC_Financial ).Should().Be(true);

### FX

The computed value for the FX change of the LC is

In [ ]:
var computedDeltaLC_Fx = Test.GetScopes<LcChangeInEstimate>(identities).Aggregate()
                                    .Fx.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [ ]:
computedDeltaLC_Fx

This value can be independently computed starting from the $\Delta\text{LC}$, and considering only the FX term

In [ ]:
var expectedDeltaLC_Fx = deltaLC.Filter(("VariableType", AocTypes.FX)).Aggregate().Value;

In [ ]:
expectedDeltaLC_Fx

In [ ]:
expectedDeltaLC_Fx.CheckEquality( -computedDeltaLC_Fx ).Should().Be(true);

## Loss Recovery Component

### Non Financial

If the [target Group of Contract](#report) is not of type Re-Insurance, the result of the following will return <code>null</code>.

The computed value for the non financial change of the LoReCo is

In [ ]:
var computedDeltaLORECO_nonFinancial = Test.GetScopes<LorecoChangeInEstimate>(identities).Aggregate()
                                           .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
computedDeltaLORECO_nonFinancial

This value can be independently computed starting from the full LoReCo data from which the delta can be computed:

In [ ]:
var LORECO = Test.GetScopes<Loreco>(identities).Aggregate().Loreco.Filter(("GroupOfContract", dataNode));

In [ ]:
var deltaLORECO = LORECO.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                  LORECO.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [ ]:
deltaLORECO.Aggregate()?.Value?? 0

The non Financial contribution is calculated subtracting the Amortization and the Interest Accretion terms:

In [ ]:
var expectedDeltaLORECO_nonFinancial = deltaLORECO.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX")).Aggregate()?.Value?? 0;

In [ ]:
expectedDeltaLORECO_nonFinancial

In [ ]:
expectedDeltaLORECO_nonFinancial.CheckEquality( -computedDeltaLORECO_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the LoReCo is

In [ ]:
var computedDeltaLORECO_Financial = Test.GetScopes<LorecoChangeInEstimate>(identities).Aggregate()
                                    .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
computedDeltaLORECO_Financial

This value can be independently computed starting from the $\Delta\text{LoReCo}$, and considering only the Interest Accretion, Yield Curve Update, Credit Risk Update terms

In [ ]:
var expectedDeltaLORECO_Financial = ( deltaLORECO.Filter(("VariableType", "IA")) +
                                     deltaLORECO.Filter(("VariableType", "YCU")) +
                                     deltaLORECO.Filter(("VariableType", "CRU")) )
                                    .Aggregate()?.Value?? 0;

In [ ]:
expectedDeltaLORECO_Financial

In [ ]:
expectedDeltaLORECO_Financial.CheckEquality( -computedDeltaLORECO_Financial ).Should().Be(true);

### FX

The computed value for the FX change of the LoReCo is

In [ ]:
var computedDeltaLORECO_Fx = Test.GetScopes<LorecoChangeInEstimate>(identities).Aggregate()
                                    .Fx.Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
computedDeltaLORECO_Fx

This value can be independently computed starting from the $\Delta\text{LoReCo}$, and considering only the FX term

In [ ]:
var expectedDeltaLORECO_Fx = deltaLORECO.Filter(("VariableType", AocTypes.FX)).Aggregate()?.Value?? 0;

In [ ]:
expectedDeltaLORECO_Fx

In [ ]:
expectedDeltaLORECO_Fx.CheckEquality( -computedDeltaLORECO_Fx ).Should().Be(true);

<br/><br/>
<a name='reconciliation'></a>
# Reconciliation

Based on the results calculated in Section 3, we check here the consistency of the [Financial Performance Report](#report). 

Finer granular reconciliations can be performed by following the datails of how each report contributes to the Financial Performance Report. Please, refer to the [Report documentation](https://portal.stage.systemorph.cloud/project/ifrs17ce/env/dev/Report/ReportScopes#financial-performance) for all the details. 

## Insurance Revenue

The Insurance Revenue (IR) contributions vary from Insurance to Re-Insurance and depend on the Liability Type of the [target Group of Contract](#report). This can be summarized by the formulas below

$$
\text{IR} = \left\{ 
\begin{array}{ll}
\Delta\text{FCF Gross Non Financial} + \Delta\text{CSM Non Financial} + \Delta\text{CSM Release} \\
+ \text{Incurred Incoming Cashflows} + \text{Claims ICO} + \text{Incurred Deferrals} \\
+ \text{Exc. Experience Adjustment on Premiums} ~~,
& \text{for LRC Insurance contracts} \\[0.2cm]
\Delta\text{CSM Non Financial} + \Delta\text{CSM Release} \\
+ \text{Incurred Incoming Cashflows} + \text{Claims ICO} + \text{Incurred Deferrals} ~~,
& \text{for LIC Insurance contracts} \\[0.2cm]
\text{Claims ICO} + \text{Incurred Deferrals} ~~,
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the IR term:

In [ ]:
var expectedDeltaCSM_release = deltaCSM.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredIncomingCashflows = Test.GetScopes<IncurredActuals>(identities).Aggregate().Premiums
                                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredDeferralsIr = Test.GetScopes<IncurredDeferrals>(identities).Aggregate().AmortizationToIr
                                      .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedClaimsICO = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsIcoToIr
                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedExcExperienceAdjPremiums = Test.GetScopes<ExperienceAdjustmentOnPremium>(identities).Aggregate().ExperienceAdjustmentOnPremium
                                           .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
double IR = 0;
if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LRC")
    IR = - expectedDeltaFCF_nonFinancial 
         - expectedDeltaCSM_nonFinancial 
         - expectedDeltaCSM_release 
         + expectedIncurredIncomingCashflows
         + expectedClaimsICO
         + expectedIncurredDeferralsIr
         + expectedExcExperienceAdjPremiums;
else if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LIC")
    IR = - expectedDeltaCSM_nonFinancial
         - expectedDeltaCSM_release
         + expectedIncurredIncomingCashflows
         + expectedClaimsICO
         + expectedIncurredDeferralsIr 
         + expectedExcExperienceAdjPremiums;
else if(groupOfContract.Partner != null)
    IR = + expectedClaimsICO
         + expectedIncurredDeferralsIr
         + expectedExcExperienceAdjPremiums;

In [ ]:
IR

## Insurance Service Expenses

Analogously, the Insurance Service Expenses (ISE) read

$$
\text{ISE} = \left\{ 
\begin{array}{ll}
\Delta\text{LC Non Financial}
+ \Delta\text{LC Release}
+ \text{Incurred ClaimsNIC} \\
+ \text{Incurred Expenses}
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for LRC Insurance contracts} \\[0.2cm]
\Delta\text{FCF Reinsurance Non Financial}
+ \Delta\text{LC Non Financial}
+ \Delta\text{LC Release}
+ \text{Incurred ClaimsNIC} \\
+ \text{Incurred Expenses} 
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for LIC Insurance contracts} \\[0.2cm]
\Delta\text{FCF Reinsurance Non Financial} 
+ \Delta\text{CSM Non Financial} 
+ \Delta\text{CSM Release} \\
+ \Delta\text{LC Non Financial} 
+ \Delta\text{LC Release} 
+ \Delta\text{LoReCo Non Financial} \\
+ \text{Incurred Incoming Cashflows} 
+ \text{Incurred ClaimsNIC} 
+ \text{Incurred Expenses} \\
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the ISE term:

In [ ]:
var expectedDeltaLC_release = deltaLC.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [ ]:
var expectedDeltaLoReCo_release = deltaLORECO.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredClaimsNIC = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsNic
                                    .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredExpenses = Test.GetScopes<IncurredActuals>(identities).Aggregate().Expenses
                                   .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredCommissions = Test.GetScopes<IncurredActuals>(identities).Aggregate().Commissions
                                      .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredClaimsICO = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsIcoToIse
                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
var expectedIncurredDeferralsIse = Test.GetScopes<IncurredDeferrals>(identities).Aggregate().AmortizationToIse
                                       .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [ ]:
double ISE = 0;
if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LRC")
    ISE = - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;
else if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LIC")
    ISE = - expectedDeltaFCF_nonFinancial
          - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;
else if(groupOfContract.Partner != null)
    ISE = - expectedDeltaFCF_nonFinancial
          - expectedDeltaCSM_nonFinancial
          - expectedDeltaCSM_release
          - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          - expectedDeltaLoReCo_release
          + expectedIncurredIncomingCashflows
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;

In [ ]:
ISE

## Insurance Finance Income/Expenses

The Financial part associated to the Insurance business encompassing both the Income and Expenses is denoted IFIE from the initials marked in capital letters. The  

$$
\text{IFIE} = \left\{
\begin{array}{ll}
\Delta\text{FCF Financial} + \Delta\text{CSM Financial} + \Delta\text{LC Financial} + \Delta\text{FCF FX} + \Delta\text{CSM FX} + \Delta\text{LC FX} ~~,
& \text{for Insurance contracts} \\[0.2cm]
\Delta\text{FCF Financial} + \Delta\text{CSM Financial} + \Delta\text{LC Financial} 
+ \Delta\text{LoReCo Financial} + \Delta\text{FCF FX} + \Delta\text{CSM FX} + \Delta\text{LC FX} 
+ \Delta\text{LoReCo FX} ~~, 
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the IFIE term:

In [ ]:
double IFIE = 0;
if(groupOfContract.Partner == null) 
    IFIE = - expectedDeltaFCF_Financial 
           - expectedDeltaCSM_Financial
           - expectedDeltaLC_Financial
           - expectedDeltaFCF_Fx 
           - expectedDeltaCSM_Fx
           - expectedDeltaLC_Fx;
else
    IFIE = - expectedDeltaFCF_Financial 
           - expectedDeltaCSM_Financial
           - expectedDeltaLC_Financial
           - expectedDeltaLORECO_Financial
           - expectedDeltaFCF_Fx 
           - expectedDeltaCSM_Fx
           - expectedDeltaLC_Fx
           - expectedDeltaLORECO_Fx;

In [ ]:
IFIE

## Other Comprehensive Income

The Other Comprehensive Income (OCI) can be non-zero only when the OCI is enabled for [target Group of Contract](#report), that is, <code>Oci Type = Default</code>.

$$
\text{OCI} = \Delta\text{FCF} \big|_{\text{Locked-in}} - \Delta\text{FCF} \big|_{\text{Current}} ~~.
$$

Here below we recompute the OCI term:

In [ ]:
var OCI = expectedDeltaFCF_OCI + expectedFxFCF_OCI;

In [ ]:
OCI